這裡開始已經假設你已經看過前面的所有基礎文件說明，因此多數註解會拿掉以維護版面乾淨

LeNet-5為深度學習界的Hello World，這意味著如果你想用深度學習做電腦視覺的事情，第一個範例就一定是它，論文滿滿的46頁，雖然有些觀念與現在相比已經有所落差，但基礎的思想是不變的，也一定要看過。

在下已有翻譯LeNet-5論文，也可以參閱[相關文件](https://hackmd.io/@shaoeChen/SyjI6W2zB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FrJvD_alOS)

首先載入相關需求套件

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.1.0'

指定硬體資源

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

取得資料集MNIST並做相關前置處理

In [4]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train / 255., -1)
x_test = np.expand_dims(x_test / 255., -1)

確認資料維度

In [5]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

將照片做padding，與論文同步的作法，這可以簡單的利用`np.pad`來達成

In [6]:
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)))

padding之後確認資料維度

In [7]:
x_train.shape

(60000, 32, 32, 1)

定義dataset

In [8]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [9]:
datasets = datasets.shuffle(buffer_size=10000, seed=10) \
                   .batch(128) \
                   .repeat() \
                   .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

從論文我們可以知道，整個LeNet-5的架構非常簡單，但要注意幾個特殊點：
1. 當初是使用average pooling而不是max pooling
2. 在layer3的時候並沒有full connect，而單純只有10個feature map連接
3. 使用tanh而不是sigmoid
4. input的部份有padding出去4個pixel，這是為了讓filter可以將邊角取值在中間而做

精神有到，但架構這種東西始終是要依實際情況調整

利用標準的keras Sequential來建置模型

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(32, 32, 1)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='tanh'),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

確認模型

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

編譯模型

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

訓練模型

In [13]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 4s 8ms/step - loss: 0.9291 - accuracy: 0.7783
Epoch 2/5
468/468 [==============================] - 2s 5ms/step - loss: 0.3017 - accuracy: 0.9177
Epoch 3/5
468/468 [==============================] - 2s 4ms/step - loss: 0.2051 - accuracy: 0.9428
Epoch 4/5
468/468 [==============================] - 2s 4ms/step - loss: 0.1553 - accuracy: 0.9557
Epoch 5/5
468/468 [==============================] - 2s 4ms/step - loss: 0.1258 - accuracy: 0.9641
CPU times: user 19.1 s, sys: 2.26 s, total: 21.4 s
Wall time: 12.1 s


簡單的訓練五次的迭代就有不錯的效果，MNIST這個資料集是電腦視覺中的一個很重要的資料集，很多論文的驗證都少不了它，記得善加利用。